# Inference with Roberta large
by Artyom Glazunov

Here you can find an example of the inference using Roberta large model. The tokenizer and initial model are loaded from https://www.kaggle.com/artemglazunov1990/roberta-save, a finetuned model you can load from your notebook (one example you can find here https://www.kaggle.com/artemglazunov1990/transfer-learning-with-roberta-large). To do that you can load outputs of your notebook with saved models in the inference notebook as an input (to save the model you can simply save its state_dict (https://pytorch.org/tutorials/beginner/saving_loading_models.html) and then load it as it is done here). 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%bash
cp ../input/roberta-save/rob.zip .
cp ../input/roberta-save/rob_tok.zip .
unzip rob.zip
unzip rob_tok.zip 
rm -r rob.zip rob_tok.zip 

In [ ]:
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch
from torch.utils.data import TensorDataset, DataLoader
import gc

Load test data

In [ ]:
df_test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
df_test

In [ ]:
data_test = df_test['excerpt'].values
ind_test = df_test['id'].values
data_test[0], ind_test[0]

In [ ]:
ind_test[df_test.index]

Load the tokenizer from another notebook's output

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(
    'rob_tok'
)

Prepare our data for the Roberta input

In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    data_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
ids_test_tensor = torch.tensor(df_test.index, dtype=torch.float)

In [ ]:
dataset_test = TensorDataset(input_ids_test,
                            attention_masks_test,
                            ids_test_tensor)

In [ ]:
batch_size = 8

dataloader_test = DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle = False
)

Our GPU device to use Roberta on

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Our model class, it uses pretrained initial Roberta model

In [ ]:
class BERTRegressor(torch.nn.Module): 
    def __init__(self, pretrained_src = 'rob'): 
        super().__init__()
        self.bert = RobertaModel.from_pretrained(pretrained_src)
        self.linear = torch.nn.Linear(1024, 1)
        self.dropout = torch.nn.Dropout(0.15)
        
    def forward(self, input_ids, attention_mask): #x - tokenized batch
        hidden = self.bert(input_ids, 
                           attention_mask=attention_mask)[0][:, 0, :]#CLS token output                                                          
        output = self.linear(self.dropout(hidden))
        return output


Load our finetuned state dict from another notebook

In [ ]:
model = BERTRegressor()
PATH = '...' #here will be you path to the model's state dict
model.load_state_dict(torch.load(PATH), strict=False)
model.to(device);

Let's define our function for prediction

In [ ]:
def predict(dataloader_test, model):

    model.eval()
    
    predictions, indices_all = [], []
    
    for batch in tqdm.notebook.tqdm(dataloader_test):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
        }
        indices_batch = batch[2].cpu().numpy()

        with torch.no_grad():        
            output = model(**inputs)
            
        output = output.detach().cpu().numpy()
        predictions.append(output)
        indices_all.append(indices_batch)    
      
    predictions = np.concatenate(predictions, axis=0)
    indices = np.concatenate(indices_all, axis=0)
            
    return predictions, indices.astype(int)

In [ ]:
predictions, indices = predict(dataloader_test, model)  

In [ ]:
predictions, indices

Clear the memory

In [ ]:
model.cpu();
del model
gc.collect()

Final submission

In [ ]:
ids = pd.DataFrame(ind_test[indices],columns=['id'])['id']

In [ ]:
sub_df = pd.DataFrame(predictions, index=ids,
                      columns = ['target'])
sub_df

In [ ]:
sub_df.to_csv('submission.csv')

In [ ]:
!rm -r rob rob_tok